# Ch09. 순환 신경망 (Recurrent Neural Network)

# v05. RNN을 이용한 텍스트 생성 (Text Generation using RNN)

- 이번 챕터에서는 **다 대 일(many-to-one)** 구조의 RNN을 사용하여 문맥을 반영해서 텍스트를 생성하는 모델을 만들어 본다.

<br>

## 5.1 RNN을 이용하여 텍스트 생성하기

- 다음과 같이 3 가지의 문장이 있다고 하자.

> "경마장에 있는 말이 뛰고 있다"

> "그의 말이 법이다"

> "가는 말이 고와야 오는 말이 곱다"

- 모델이 문맥을 학습할 수 있도록 전체 문장의 앞의 단어들을 전부 고려하여 학습하도록 데이터를 재구성한다면 아래와 같이 총 11개의 샘플이 구성된다.

| samples | $X$                        | $y$    |
| :------ | :------------------------- | :----- |
| 1       | 경마장에                   | 있는   |
| 2       | 경마장에 있는              | 말이   |
| 3       | 경마장에 있는 말이         | 뛰고   |
| 4       | 경마장에 있는 말이 뛰고    | 있다   |
| 5       | 그의                       | 말이   |
| 6       | 그의 말이                  | 법이다 |
| 7       | 가는                       | 말이   |
| 8       | 가는 말이                  | 고와야 |
| 9       | 가는 말이 고와야           | 오는   |
| 10      | 가는 말이 고와야 오는      | 말이   |
| 11      | 가는 말이 고와야 오는 말이 | 곱다   |

<br>

### 5.1.1 데이터에 대한 이해와 전처리

#### 5.1.1.1 필요 라이브러리 임포트

In [1]:
%tensorflow_version 2.x

import tensorflow as tf
tf.__version__

TensorFlow 2.x selected.


'2.1.0'

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

<br>

#### 5.1.1.2 학습 데이터

- 예제로 언급한 3개의 한국어 문장을 저장한다.

In [ ]:
text = """경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다"""

In [4]:
text

'경마장에 있는 말이 뛰고 있다\n그의 말이 법이다\n가는 말이 고와야 오는 말이 곱다'

<br>

#### 5.1.1.3 단어 집합 생성

- 단어 집합을 생성하고 크기를 확인해보자.

In [5]:
t = Tokenizer()
t.fit_on_texts([text])

vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스의 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야 하므로 미리 +1 선언

print("단어 집합의 크기 : %d" % vocab_size)

단어 집합의 크기 : 12


<br>

- 각 단어와 단어에 부여된 정수 인덱스를 출력

In [6]:
t.word_index

{'가는': 8,
 '경마장에': 2,
 '고와야': 9,
 '곱다': 11,
 '그의': 6,
 '뛰고': 4,
 '말이': 1,
 '법이다': 7,
 '오는': 10,
 '있는': 3,
 '있다': 5}

<br>

#### 5.1.1.4 훈련 데이터 생성

In [7]:
sequences = list()

for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]

    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수 : %d' % len(sequences))

학습에 사용할 샘플의 개수 : 11


<br>

- 전체 샘플 출력

In [8]:
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

- 위의 데이터는 아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터이다.
- `[2, 3]` : `[경마장에, 있는]`
- `[2, 3, 1]` : `[경마장에, 있는, 말이]`
- 전체 훈련 데이터에 대해서 맨 우측에 있는 단어에 대해서만 레이블로 분리해야 한다.

<br>

#### 5.1.1.5 훈련 데이터 패딩(padding)

- 우선 전체 샘플에 대해서 길이를 일치시켜 준다.
- 가장 긴 샘플의 길이를 기준으로 한다.
- 현재 눈으로 봤을 때, 가장 길이가 긴 샘플은 `[8, 1, 9, 10, 1, 11]`이고 길이는 6이다.
- 이를 코드로는 다음과 같이 구할 수 있다.

In [9]:
max_len = max(len(l) for l in sequences)

print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


<br>

- 전체 샘플의 길이를 6으로 패딩한다.

In [ ]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

- `pad_sequences()`는 모든 샘플에 대해서 0을 사용하여 길이를 맞춘다.
- `maxlen`의 값으로 6을 주면 모든 샘플의 길이를 6으로 맞춰준다.
- `padding`의 인자로 `pre`를 주면 길이가 6보다 짧은 샘플의 앞에 0으로 채운다.

<br>

- 전체 훈련 데이터 출력

In [11]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


- 길이가 6보다 짧은 모든 샘플에 대해서 앞에 9을 채워서 모든 샘플의 길이를 6으로 바꿨다.

<br>

#### 5.1.1.6 레이블 분리

- 이제 각 샘플의 마지막 단어를 레이블로 분리한다.
- 레이블의 분리는 Numpy를 이용해서 가능하다.

In [ ]:
sequences = np.array(sequences)

X = sequences[:, :-1] # 리스트의 마지막 값을 제외하고 저장한 것
y = sequences[:, -1] # 리스트의 마지막 값만 저장한 것 (레이블)

<br>

- 분리된 X와 y 출력

In [13]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [14]:
print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


<br>

#### 5.1.1.7 레이블 원-핫 인코딩

- 이제 RNN 모델에 훈련 데이터를 훈련 시키기 전에 레이블에 대해서 원-핫 인코딩을 수행한다.

In [ ]:
y = to_categorical(y, num_classes=vocab_size)

In [16]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


<br>

### 5.1.2 모델 설계하기

- 이제 RNN 모델에 데이터를 훈련시킨다.

<br>

#### 5.1.2.1 필요 라이브러리 임포트

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

<br>

#### 5.1.2.2 모델 설계 및 훈련

- 각 단어의 임베딩 벡터는 10차원을 가짐
- 32의 은닉 상태를 가지는 바닐라 RNN을 사용

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) # 레이블을 분리하였으므로 이제 X의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, y, epochs=200, verbose=2)

```python
Train on 11 samples
Epoch 1/200
11/11 - 1s - loss: 2.4856 - accuracy: 0.0000e+00
Epoch 2/200
11/11 - 0s - loss: 2.4748 - accuracy: 0.0000e+00
Epoch 3/200
11/11 - 0s - loss: 2.4640 - accuracy: 0.0000e+00

...

Epoch 198/200
11/11 - 0s - loss: 0.0845 - accuracy: 1.0000
Epoch 199/200
11/11 - 0s - loss: 0.0831 - accuracy: 1.0000
Epoch 200/200
11/11 - 0s - loss: 0.0817 - accuracy: 1.0000
<tensorflow.python.keras.callbacks.History at 0x7f94903de6a0>
```

<br>

#### 5.1.2.3 모델 평가

- 모델이 정확하게 예측하고 있는 지 문장을 생성하는 함수를 만들어서 출력해보자

In [ ]:
def sentence_generation(model, t, current_word, n):
    # model : 모델, t : 토크나이저, current_word : 현재 단어, n : 반복할 횟수

    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기 위해 저장
    sentence = ''

    for _ in range(n):

        # 현재 단어에 대한 정수 인코딩
        encoded = t.texts_to_sequences([current_word])[0]

        # 데이터에 대한 패딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')

        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장
        result = model.predict_classes(encoded, verbose=0)

        for word, index in t.word_index.items():
            if index == result: # 만약 예측한 단어와 동일한 인덱스와 동일한 단어가 있다면
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' ' + word 

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence

    return sentence

In [22]:
# '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측
print(sentence_generation(model, t, '경마장에', 4))

경마장에 있는 말이 뛰고 있다


In [23]:
print(sentence_generation(model, t, '그의', 2)) # 2번 예측

그의 말이 법이다


In [24]:
print(sentence_generation(model, t, '가는', 5)) # 5번 예측

가는 말이 고와야 오는 말이 곱다


- 이제 앞의 문맥을 기준으로 '말이' 라는 단어 다음에 나올 단어를 기존의 훈련 데이터와 일치하게 예측함을 보여준다.

<br>

#### 5.1.2.4 모델의 한계

- 이 모델은 충분한 훈련 데이터를 갖고 있지 못하므로 위에서 문장의 길이에 맞게 적절하게 예측해야 하는 횟수 4, 2, 5를 각각 인자값으로 주었다.
- 이 이상의 숫자를 주면 기계는 '있다', '법이다', '곱다' 다음에 나오는 단어가 무엇인 지 배운 적이 없으므로 임의 예측을 한다.
- 이제 더 많은 훈련 데이터를 가지고 실습을 해보자.

<br>

## 5.2 LSTM을 이용하여 텍스트 생성하기

- 이번에는 LSTM을 통해 보다 많은 데이터로 텍스트를 생성해보자.
- 본질적으로 앞에서 한 것과 동일한 실습이다.

<br>

### 5.2.1 데이터에 대한 이해와 전처리

#### 5.2.1.1 사용할 데이터

- 뉴욕 타임즈 기사의 제목
- [해당 링크](https://www.kaggle.com/aashita/nyt-comments)에서 `ArticlesApril2018.csv` 데이터 다운로드

<br>

#### 5.2.1.2 필요 라이브러리 임포트

In [ ]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

<br>

#### 5.2.1.3 훈련 데이터 데이터프레임에 저장

In [27]:
df = pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


<br>

#### 5.2.1.4 훈련 데이터의 컬럼 확인

- 어떤 열이 있고, 열이 총 몇 개가 있는 지 확인

In [28]:
print('열의 개수 : ', len(df.columns))

열의 개수 :  15


In [30]:
print(df.columns)

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


- 총 15개의 열이 존재
- 여기서 사용할 열은 제목에 해당되는 `headline`이다.

<br>

#### 5.2.1.5 결측값 확인

In [31]:
df['headline'].isnull().values.any()

False

- Null 값은 별도로 없는 것 확인

<br>

#### 5.2.1.6 노이즈 데이터 확인

- `headline` 열에서 모든 신문 기사의 제목을 뽑아서 하나의 리스트로 저장

In [32]:
headline = []

headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

- 4번 째, 5번 째 샘플에 `Unknown` 값이 들어가 있다.
- `headline` 전체에 걸쳐서 `Unknown` 값을 가진 샘플이 있을 것으로 추정된다.
- 비록 Null 값은 아니지만 지금 하고자 하는 실습에 도움이 되지 않는 노이즈 데이터이므로 제거해 줄 필요가 있다.

- 제거하기 전에 현재 샘플의 개수를 확인해보고 제거 전, 후의 샘플의 개수를 비교해보자.

In [33]:
print('총 샘플의 개수 : {}'.format(len(headline)))

총 샘플의 개수 : 1324


In [35]:
headline = [n for n in headline if n != "Unknown"]
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline)))

노이즈값 제거 후 샘플의 개수 : 1214


- 샘플의 수가 1,324에서 1,214로 110개의 샘플이 제거됨

- 기존에 출력했던 5개의 샘플을 출력

In [36]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

- 기존에 4번째, 5번째 샘플에서는 Unknown 값이 있었는데 현재는 제거가 된 것을 확인

<br>

#### 5.2.1.7 데이터 전처리 수행

- 여기서 선택한 전처리
  - 구두점 제거
  - 단어의 소문자화

- 전처리를 수행하고 다시 샘플 5개를 출력

In [ ]:
def repreprocessing(s):
    s = s.encode("utf8").decode("ascii", "ignore")

    return ''.join(c for c in s if c not in punctuation).lower()

In [38]:
text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

- 기존의 출력과 비교
  - 모든 단어들이 소문자화됨
  - "N.F.L"이나 "Cheerleaders'" 등과 같이 기존에 구두점이 붙어있던 단어들에서 구두점이 제거됨

<br>

#### 5.2.1.8 단어 집합(vocabulary) 생성

- 단어 집합을 만들고 크기를 확인

In [40]:
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.index_word) + 1

print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


- 총 3,494개의 단어가 존재한다.

<br>

#### 5.2.1.9 정수 인코딩 및 훈련 데이터 구성

- 이제 정수 인코딩과 동시에 하나의 문장을 여러 줄로 분해하여 훈련 데이터를 구성한다.